In [2]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2,stride=2)
        self.conv2 = nn.Conv2d(5, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [5]:
net = Net()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001,
                            momentum=0.9)

In [7]:
train_dataset = dsets.CIFAR10(root="./datasets/",
                             train=True,
                             transform=transforms.ToTensor(),
                             download=False)

In [8]:
test_dataset = dsets.CIFAR10(root="./datasets/",
                            train=False,
                            transform=transforms.ToTensor(),
                            download=False)

In [9]:
batch_size = 64

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [11]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [15]:
for epoch in range(2):
    running_loss = 0.
    for i, (images,labels) in enumerate(train_loader):
        # wrap the inputs into Variable
        # enables autograd function
        inputs = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i%2000 == 0:
            print("[%d/%5d] loss: %.3f" 
                 % (epoch+1, i+1, running_loss/2000 ))
            running_loss = 0.0
print("Finished training")

[1/    1] loss: 0.001
[2/    1] loss: 0.001
Finished training
